In [1]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("merged_train", {}, r"/home/main/Desktop/detectron2/datasets/merged_oc/annotations/train.json", r"/home/main/Desktop/detectron2/datasets/merged_oc/images")
register_coco_instances("merged_val", {}, r"/home/main/Desktop/detectron2/datasets/merged_oc/annotations/val.json", r"/home/main/Desktop/detectron2/datasets/merged_oc/images/")


In [2]:
from detectron2.data import DatasetCatalog, MetadataCatalog
keypoint_names = ["front"]
keypoint_flip_map = []
MetadataCatalog.get("merged_train").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)
MetadataCatalog.get("merged_val").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)

namespace(name='merged_val',
          json_file='/home/main/Desktop/detectron2/datasets/merged_oc/annotations/val.json',
          image_root='/home/main/Desktop/detectron2/datasets/merged_oc/images/',
          evaluator_type='coco',
          keypoint_names=['front'],
          keypoint_flip_map=[])

In [3]:
from detectron2.config import get_cfg
from detectron2 import model_zoo

modelname = "maskrcnn"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("merged_train",)
cfg.DATASETS.TEST = ("merged_val",)
cfg.OUTPUT_DIR = r"/home/main/Desktop/detectron2/manipulator/evaluation/maskrcnn"


cfg.DATALOADER.NUM_WORKERS = 4
#cfg.MODEL.WEIGHTS = r"/home/main/Desktop/detectron2/manipulator/checkpoints/combined_checkpoint.pkl"
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (1000, 1500)

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.NUM_INSTANCES_PER_IMAGE = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85   # set a custom testing threshold


cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE = 1

cfg.TEST.EVAL_PERIOD = cfg.SOLVER.MAX_ITER // 20
cfg.TEST.KEYPOINT_OKS_SIGMAS = (1,)
cfg.TEST.DETECTIONS_PER_IMAGE = 8
cfg.TEST.AUG.ENABLED = True

print(cfg)
# save config to file
cfg.dump(cfg.OUTPUT_DIR + "/config.yaml")

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('merged_val',)
  TRAIN: ('merged_train',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32], [64], [128], [256], [512]]
  BACKBONE:
    FREEZE_AT: 2
    NAME: build_resnet_fpn_backbone
  DEVICE: cuda
  FPN:
    FUSE_TYPE: sum
    IN_FEATURES: ['res2', 'res3'

In [7]:
## PointRend
from detectron2.projects.point_rend.config import add_pointrend_config
from detectron2.projects.point_rend.mask_head import PointRendMaskHead
from detectron2.config import get_cfg
from detectron2 import model_zoo

modelname = "pointrend"

cfg = get_cfg()
#cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#
add_pointrend_config(cfg)
cfg.merge_from_file("/home/main/Desktop/detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco.yaml")
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))


cfg.DATASETS.TRAIN = ("merged_train",)
cfg.DATASETS.TEST = ("merged_val",)
cfg.OUTPUT_DIR = r"/home/main/Desktop/detectron2/manipulator/evaluation/pointrend"

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = r"/home/main/Desktop/detectron2/manipulator/checkpoints/combined_checkpoint.pkl"
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 1000
#cfg.SOLVER.STEPS = (1000,)


cfg.MODEL.ROI_MASK_HEAD.NAME = "PointRendMaskHead"
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.NUM_INSTANCES_PER_IMAGE = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85   # set a custom testing threshold


cfg.MODEL.POINT_HEAD.NUM_CLASSES = cfg.MODEL.ROI_HEADS.NUM_CLASSES

cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE = 1

cfg.TEST.EVAL_PERIOD = 25
cfg.TEST.KEYPOINT_OKS_SIGMAS = (1,)
cfg.TEST.DETECTIONS_PER_IMAGE = 8
cfg.TEST.AUG.ENABLED = True
cfg.TEST.OUTPUT_DIR = r"/home/main/Desktop/detectron2/manipulator/evaluation/pointrend"


In [4]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
import os

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs(os.path.join(cfg.OUTPUT_DIR, "coco_eval"), exist_ok=True)
            output_folder = os.path.join(cfg.OUTPUT_DIR, "coco_eval")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

#trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[07/01 00:05:26 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Some model parameters or buffers are not found in the checkpoint:
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.keypoint_head.conv_fcn1.{bias, weight}
roi_heads.keypoint_head.conv_fcn2.{bias, weight}
roi_heads.keypoint_head.conv_fcn3.{bias, weight}
roi_heads.keypoint_head.conv_fcn4.{bias, weight}
roi_heads.keypoint_head.conv_fcn5.{bias, weight}
roi_heads.keypoint_head.conv_fcn6.{bias, weight}
roi_head

[07/01 00:05:26 d2.engine.train_loop]: Starting training from iteration 0


/home/main/miniconda3/lib/python3.10/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  keypoints = torch.as_tensor(keypoints, dtype=torch.float32, device=device)
/home/main/miniconda3/lib/python3.10/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  keypoints = torch.as_tensor(keypoints, dtype=torch.float32, device=device)
/home/main/miniconda3/lib/python3.10/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays

[07/01 00:06:10 d2.utils.events]:  eta: 1:15:50  iter: 19  total_loss: 11.58  loss_cls: 0.825  loss_box_reg: 0.155  loss_mask: 0.694  loss_keypoint: 8.801  loss_rpn_cls: 0.6733  loss_rpn_loc: 0.3657    time: 2.1848  last_time: 2.3459  data_time: 0.0413  last_data_time: 0.0265   lr: 5.762e-06  max_mem: 6056M
[07/01 00:06:55 d2.utils.events]:  eta: 1:15:02  iter: 39  total_loss: 9.998  loss_cls: 0.4282  loss_box_reg: 0.1773  loss_mask: 0.6931  loss_keypoint: 7.588  loss_rpn_cls: 0.6732  loss_rpn_loc: 0.3956    time: 2.2152  last_time: 2.2996  data_time: 0.0284  last_data_time: 0.0271   lr: 9.722e-06  max_mem: 6056M
[07/01 00:07:40 d2.utils.events]:  eta: 1:14:11  iter: 59  total_loss: 9.24  loss_cls: 0.3528  loss_box_reg: 0.2147  loss_mask: 0.6903  loss_keypoint: 6.949  loss_rpn_cls: 0.6642  loss_rpn_loc: 0.4012    time: 2.2189  last_time: 2.2451  data_time: 0.0299  last_data_time: 0.0397   lr: 1.3682e-05  max_mem: 6056M
[07/01 00:08:24 d2.utils.events]:  eta: 1:13:25  iter: 79  total_lo

KeyboardInterrupt: 

In [5]:
import torch
torch.save(trainer.model, os.path.join(cfg.OUTPUT_DIR, 'checkpoint.pth'))
torch.save(trainer.model.state_dict(), os.path.join(cfg.OUTPUT_DIR, "checkpoint_state_dict.pth"))


In [6]:
import os
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultPredictor

val_dataset_path = r"/home/main/Desktop/detectron2/datasets/testing/"
evaluation_dataset_path = os.path.join(cfg.OUTPUT_DIR, "inference")

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "checkpoint_state_dict.pth")

predictor = DefaultPredictor(cfg)
for root, dirs, files in os.walk(val_dataset_path):
    folder = root.split('/')[-1]
    if not os.path.exists(os.path.join(evaluation_dataset_path, folder)):
        os.makedirs(os.path.join(evaluation_dataset_path, folder))
    for image in files:
        print(image)
        img = cv2.imread(os.path.join(root, image))
        outputs = predictor(img)
        v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        out = v.get_image()[:, :, ::-1]
        cv2.imwrite(os.path.join(evaluation_dataset_path, folder, image), out)


[06/30 17:58:17 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/main/Desktop/detectron2/manipulator/evaluation/maskrcnn/checkpoint_state_dict.pth ...
image_006.tif
Xilinx_n1_109.tif
image_22nm_008.tif
image_073.tif
image_Ne cut_006.tif
image_074.tif
image_22nm_010.tif
22nm_018.tif
Xilinx_n1_108.tif
image_019.tif
image_090.tif
image_014.tif
image_017.tif
image_012.tif
image_079.tif
image_058.tif
image_Ne cut_007.tif
14 nm_024.tif
image_071.tif
Image012.tif
image_081.tif
image_088.tif
image_011.tif
image_004.tif
image_005.tif
image_009.tif
A14_4_009.tif
image_22nm_001.tif
image_002.tif
driftmessung_007.tif
Marvell_tran_003.tif
Apple_A14_ebic_003_EBIC.tif
Marvell_tran_001.tif
image_073.tif
TSMC 7nm_ebic_018_EBIC.tif
image_003.tif
image__005.tif
image_097.tif
Apple_A14_Nr4_keithley_IdsVds_001.tif
Murata_A-1-1_EBIV_ebic_003_InLens.tif
image_067.tif
TSMC 7nm_ebic_026_EBIC.tif
image_014.tif
Apple_A14_Nr4_keithley_IdsVds_004.tif
image_062.tif
image_018.tif
Sampl

In [ ]:
import pickle
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

# Load the keypoint head using the first checkpoint file (pickle format)
model = build_model(cfg)
checkpointer = DetectionCheckpointer(model)
with open(r"C:\Users\david\projects\detectron2\manipulator\checkpoints\model_final_a6e10b.pkl", "rb") as f:
    first_checkpoint = pickle.load(f)
#print(first_checkpoint['model'].keys())


# Load the mask head parameters from the second checkpoint file (pickle format)
with open(r"C:\Users\david\projects\detectron2\manipulator\checkpoints\model_final_f10217.pkl", "rb") as f:
    second_checkpoint = pickle.load(f)

mask_head_parameters = {k: v for k, v in second_checkpoint["model"].items() if "mask" in k}

print(mask_head_parameters.keys())

for param in mask_head_parameters.keys():
    first_checkpoint["model"][param] = mask_head_parameters[param]

# Save the combined model to a final checkpoint file (pickle format)
with open("combined_checkpoint.pkl", "wb") as f:
    pickle.dump(first_checkpoint, f)